# Installing necessary libraries for web scraping

In [1]:
#Installing necessary libraries for web scraping
!pip install selenium
!pip install BeautifulSoup4

# Importing necessary libraries

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Webscraping data

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# makes a request to the web page and gets its HTML
r = requests.get(url)

# stores the HTML page in 'soup', a BeautifulSoup object
soup = BeautifulSoup(r.content)

# assigning to data frame

In [4]:
a = []
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
for link in soup.find_all('td'):
    a.append(link.get_text())
    if len(a) == 3:
        df_length = len(df)
        df.loc[df_length] = a
        a = []
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
7,M8A\n,Not assigned\n,Not assigned\n
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n"
9,M1B\n,Scarborough\n,"Malvern, Rouge\n"


# Cleaning data

In [5]:
df2 = df.replace('\n','', regex=True)
df2 = df2.drop(index = range(180,191))
df2 = df2.drop(index = (df2[(df2['Borough']=='Not assigned')].index))
df2 = df2.reset_index(drop=True)
df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Checking if there are existing Not assigned value in column, Neighborhood

In [6]:
df2.loc[df2['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [7]:
df2.shape

(103, 3)

# Installing Geocoder

In [8]:
!pip install geocoder

# Importing Geocoder

In [9]:
import geocoder

# Getting the latitude and longitude of each Neighborhood

In [10]:
lat_lang_df = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lang_df = lat_lang_df.sort_values(by=['Postal Code'])
df2 = df2.sort_values(by=['PostalCode'])

In [11]:
df2['Latitude'] = lat_lang_df['Latitude']
df2['Longitude'] = lat_lang_df['Longitude']
df2 = df2.reset_index(drop=True)
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
...,...,...,...,...,...
98,M9N,York,Weston,43.696948,-79.411307
99,M9P,Etobicoke,Westmount,43.648429,-79.382280
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.647927,-79.419750
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.602414,-79.543484


# Map visualizing the city of Toronto

In [12]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
!pip install folium
import folium

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Simplifying data by getting only those in North York

In [22]:
northyork_data = df2[df2['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.shape

(24, 5)

In [20]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North york are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North york are 43.7543263, -79.44911696639593.


In [21]:
# create map of Manhattan using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

# Exploring venues in North York Neighborhoods using Foursquare and putting it to data frame

In [23]:
CLIENT_ID = 'PB1AOIJ30RPF31TUJ5EXMHJ2A5AHPUNGRCBO3ZDWQFJ1EKGL' # your Foursquare ID
CLIENT_SECRET = 'GQBFXIWBFUHBGZSMCU3SRL1D33CXT1I5PXRGBHDUOHU1W5D1' # your Foursquare Secret
ACCESS_TOKEN = 'GQEN0N5TANV0NPNANSVMMZPYOUXOXMZUU1CL1TAQCTUO2APT' # your FourSquare Access Token
VERSION = '20210123' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PB1AOIJ30RPF31TUJ5EXMHJ2A5AHPUNGRCBO3ZDWQFJ1EKGL
CLIENT_SECRET:GQBFXIWBFUHBGZSMCU3SRL1D33CXT1I5PXRGBHDUOHU1W5D1


In [25]:
northyork_data.loc[0, 'Neighborhood']

'Hillcrest Village'

In [26]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500

Latitude and longitude values of Hillcrest Village are 43.72589970000001, -79.340923.


In [27]:
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)


In [28]:
results = requests.get(url).json()

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Sorento Restaurant,Italian Restaurant,43.726575,-79.341989
1,Fitness Connection,Gym,43.727473,-79.341707
2,Oomomo,Discount Store,43.726429,-79.343283
3,Tilley Endurables,Clothing Store,43.727033,-79.342926
4,Swiss Chalet,Restaurant,43.726747,-79.341625


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


# Using the getNearbyVenues function by neighborhoods

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'], latitudes=northyork_data['Latitude'], longitudes=northyork_data['Longitude'])

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [35]:
print(northyork_venues.shape)
northyork_venues.head()

(652, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.7259,-79.340923,Sorento Restaurant,43.726575,-79.341989,Italian Restaurant
1,Hillcrest Village,43.7259,-79.340923,Fitness Connection,43.727473,-79.341707,Gym
2,Hillcrest Village,43.7259,-79.340923,Oomomo,43.726429,-79.343283,Discount Store
3,Hillcrest Village,43.7259,-79.340923,Tilley Endurables,43.727033,-79.342926,Clothing Store
4,Hillcrest Village,43.7259,-79.340923,Swiss Chalet,43.726747,-79.341625,Restaurant


# Assessing the counts of unique Venues in each neighborhood

In [36]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,20,20,20,20,20,20
"Bedford Park, Lawrence Manor East",79,79,79,79,79,79
Don Mills,23,23,23,23,23,23
Downsview,165,165,165,165,165,165
"Fairview, Henry Farm, Oriole",4,4,4,4,4,4
Glencairn,6,6,6,6,6,6
Hillcrest Village,20,20,20,20,20,20
Humber Summit,4,4,4,4,4,4


In [37]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 170 uniques categories.


In [39]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot

,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
649,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
650,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
northyork_onehot.shape

(652, 170)

# Getting the mean of each unique venues by neighborhoods

In [42]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
1,Bayview Village,0.050000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.05,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.037975,0.000000,0.00,0.025316,0.000000,0.012658,0.00,...,0.012658,0.012658,0.000000,0.00,0.000000,0.012658,0.00000,0.000000,0.00,0.012658
3,Don Mills,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
4,Downsview,0.012121,0.000000,0.018182,0.006061,0.00,0.012121,0.000000,0.018182,0.00,...,0.000000,0.018182,0.000000,0.00,0.006061,0.006061,0.00000,0.000000,0.00,0.006061
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.166667,0.00,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.00,0.050000,0.000000,0.050000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
9,"Humberlea, Emery",0.016129,0.000000,0.000000,0.000000,0.00,0.000000,0.016129,0.000000,0.00,...,0.016129,0.000000,0.000000,0.00,0.000000,0.016129,0.00000,0.000000,0.00,0.016129


In [43]:
northyork_grouped.shape

(20, 170)

# Getting the top 5 venues in each neighborhood

In [44]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.09
1         Bank  0.09
2  Bridal Shop  0.05
3  Supermarket  0.05
4    Pet Store  0.05


----Bayview Village----
                       venue  freq
0          Indian Restaurant  0.10
1                Yoga Studio  0.05
2                       Bank  0.05
3               Burger Joint  0.05
4  Middle Eastern Restaurant  0.05


----Bedford Park, Lawrence Manor East----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2            Gastropub  0.04
3  American Restaurant  0.04
4                Hotel  0.04


----Don Mills----
          venue  freq
0      Pharmacy  0.09
1  Intersection  0.09
2      Bus Line  0.09
3   Pizza Place  0.09
4        Bakery  0.09


----Downsview----
         venue  freq
0  Coffee Shop  0.12
1        Hotel  0.05
2         Café  0.05
3       Bakery  0.03
4         Park  0.03


----Fairview, Henry Farm, Oriole----
                venue  freq